In [ ]:
!pip install kaggle


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
!kaggle datasets download -d ashery/chexpert
import zipfile

# Assuming 'chexpert.zip' is the name of the downloaded file
with zipfile.ZipFile('chexpert.zip', 'r') as zip_ref:
    zip_ref.extractall('CheXpert-v1.0')



^C
Dataset URL: https://www.kaggle.com/datasets/ashery/chexpert
License(s): CC0-1.0
chexpert.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip install pandas

In [ ]:
!pip install tensorflow

In [ ]:
!pip install setuptools


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [7]:
objects= pd.read_csv("train_visualCheXbert.csv")

In [8]:
objects.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


/content/CheXpert-v1.0/train

In [9]:
objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223414 entries, 0 to 223413
Data columns (total 19 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Path                        223414 non-null  object 
 1   Sex                         223414 non-null  object 
 2   Age                         223414 non-null  int64  
 3   Frontal/Lateral             223414 non-null  object 
 4   AP/PA                       191027 non-null  object 
 5   Enlarged Cardiomediastinum  223414 non-null  float64
 6   Cardiomegaly                223414 non-null  float64
 7   Lung Opacity                223414 non-null  float64
 8   Lung Lesion                 223414 non-null  float64
 9   Edema                       223414 non-null  float64
 10  Consolidation               223414 non-null  float64
 11  Pneumonia                   223414 non-null  float64
 12  Atelectasis                 223414 non-null  float64
 13  Pneumothorax  

In [10]:
path_df = objects["Path"]

# Prepend '/content/' to each path in the 'path_df'
path_df = path_df.str.replace('^', '/content/', regex=True)


In [11]:
path_df.head()

0    /content/CheXpert-v1.0/train/patient00001/stud...
1    /content/CheXpert-v1.0/train/patient00002/stud...
2    /content/CheXpert-v1.0/train/patient00002/stud...
3    /content/CheXpert-v1.0/train/patient00002/stud...
4    /content/CheXpert-v1.0/train/patient00003/stud...
Name: Path, dtype: object

In [12]:
# List of label columns
LABEL_COLUMNS = ["Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity",
                 "Lung Lesion", "Edema", "Consolidation", "Pneumonia",
                 "Atelectasis", "Pneumothorax", "Pleural Effusion",
                 "Pleural Other", "Fracture", "Support Devices", "No Finding"]

def preprocess_image(img_path, img_size=(224, 224)):
    # Load the image from file path, resize, and normalize it
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode JPEG image into RGB format
    img = tf.image.resize(img, img_size)  # Resize image
    img = img / 255.0  # Normalize pixel values
    return img
def load_image_and_labels(row):
    # Extract image path and labels
    img_path = row['Path']
    img = preprocess_image(img_path)

    # Extract labels as a tensor
    labels = tf.convert_to_tensor(row[LABEL_COLUMNS].values, dtype=tf.float32)

    return img, labels  # Return a tuple of (image, labels)

def create_dataset(dataframe, batch_size=32):
    # Create a TensorFlow Dataset from the dataframe
    image_paths = dataframe['Path'].values  # Extract image paths
    labels = dataframe[LABEL_COLUMNS].values.astype('float32')  # Convert labels to float32

    # Create TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Map the loading function to the dataset
    dataset = dataset.map(lambda img_path, label: (preprocess_image(img_path), label),
                          num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=len(dataframe), reshuffle_each_iteration=True)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # Pre-fetch data for better performance

    return dataset





In [16]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.8 kB 871.5 kB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB 8.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/11.0 MB 14.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/11.0 MB 14.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.7/11.0 MB 17.1 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 16.3 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.0 MB 15.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 15.3 MB/s eta 0:00:01
   ------------------- -------------------- 5.3/11.0 MB 15.6 MB/s eta 0:00:01
   ---------------------- ----------------- 6.1/11.0 MB 15.6 MB/s eta 0:00:


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

objects_filtered = objects[LABEL_COLUMNS + ['Path']].copy()

# Handle NaN values by filling them with 0 or dropping rows with NaNs
objects_filtered.fillna(0, inplace=True)


train_df, val_df = train_test_split(objects_filtered, test_size=0.3, random_state=42)

# Reset index for the new DataFrames
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)



In [14]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load a pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model layers

# Add custom classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(LABEL_COLUMNS), activation='sigmoid')  # Adjust output layer for multi-label classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
cardiomegaly_data = objects[objects['Cardiomegaly'] == 1.0]
pneumonia_data = objects[objects['Pneumonia'] == 1.0]
lung_opacity_data = objects[objects['Lung Opacity'] == 1.0]
relevant_conditions = objects[
    (objects['Cardiomegaly'] == 1.0) |
    (objects['Pneumonia'] == 1.0) |
    (objects['Lung Opacity'] == 1.0)
]

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    # First convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Third convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten the feature maps
    Flatten(),

    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting

    # Output layer for multi-label classification
    Dense(len(LABEL_COLUMNS), activation='sigmoid')  # Use 'sigmoid' for multi-label output
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


c:\Users\yadidiahk\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │         1,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,766 (42.61 MB)

 Trainable params: 11,170,766 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [17]:
train_dataset = create_dataset(train_df, batch_size=32)
val_dataset = create_dataset(val_df, batch_size=32)

train_dataset = train_dataset.cache().prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(tf.data.AUTOTUNE)


# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5, batch_size=32)


Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node resize/ResizeBilinear defined at (most recent call last):
<stack traces unavailable>
Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::MemoryCacheImpl::Prefetch::BatchV2::Shuffle::ParallelMapV2: OOM when allocating tensor with shape[1,224,224,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node resize/ResizeBilinear}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_4741]